In [1]:
import pandas as pd
import numpy as np

filelist = ['birth.csv', 'breed.csv', 'report.csv', 'spec.csv','submission.csv']

# Read File

In [2]:
birth = pd.read_csv('./data/' + filelist[0])
breed = pd.read_csv('./data/' + filelist[1])
report = pd.read_csv('./data/' + filelist[2])
spec = pd.read_csv('./data/' + filelist[3])
submission = pd.read_csv('./data/' + filelist[4])
submission

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,1
0,480,NaN
1,742,NaN
2,743,NaN
3,744,NaN
4,745,NaN
...,...,...
4258,37481,NaN
4259,37482,NaN
4260,37483,NaN
4261,37484,NaN


In [3]:
birth_index = ['乳牛編號', '分娩日期', '乾乳日期','犢牛編號1','犢牛編號2','母牛體重','登錄日期','計算胎次','胎次','分娩難易度','犢牛體型','犢牛性別','酪農場代號']
report_index = ['ID','資料年度','資料月份','酪農場代號','乳牛編號','父親精液編號','母親乳牛編號','出生日期','胎次','泌乳天數','乳量','最近分娩日期','採樣日期','月齡','檢測日期','最後配種日期','最後配種精液','配種次數','前次分娩日期','第一次配種日期','第一次配種精液']
breed_index = ['乳牛編號','配種日期','配種精液','登錄日期','孕檢','配種方式','精液種類','酪農場代號']
spec_index = ['乳牛編號','狀況類別','狀況代號','狀況日期','備註','登錄日期','酪農場代號']
birth.columns = birth_index
report.columns = report_index
breed.columns = breed_index
spec.columns = spec_index

# 建立時序資料

In [4]:
# SQL
def buildtable(data):
    t1 = data
    t2 = data
    t3 = data
    # Join
    t4 = pd.merge(t1, t2, on = ['乳牛編號', '胎次'], suffixes = ['_L', '_R'])
    # Filter
    t5 = t4[(t4.月齡_L >= t4.月齡_R)]
    # Group by
    t6 = t5.groupby(['ID_L'], as_index = False)
    # Aggregate
    t7 = t6.agg({'ID_R': 'count'})
    
    t8 = t7.rename(columns = {'ID_L': 'ID', 'ID_R': '序列號'})
    t9 = pd.merge(t8, t3, on = ['ID'])
    t10 = t9.sort_values(by = ['乳牛編號', '胎次', '序列號', '乳量'])
    return t10


# # build time series
# def build_data(train, pastMonth=5, futureMonth=5):
#     train['過去乳量-1'] = train.groupby(by=['乳牛編號', '胎次'])['乳量'].shift()
#     for i in range(2, pastMonth+1):
#         train['過去乳量-%d'%(i)] = train.groupby(by=['乳牛編號', '胎次'])['過去乳量-%d'%(i-1)].shift()
#     train['未來乳量-1'] = train.groupby(by=['乳牛編號', '胎次'])['乳量'].shift(-1)
#     for i in range(2, futureMonth+1):
#         train['未來乳量-%d'%(i)] = train.groupby(by=['乳牛編號', '胎次'])['未來乳量-%d'%(i-1)].shift(-1)
#     return train

In [5]:
report_seq = buildtable(report)

In [9]:
dc = dict()
for val in report_seq.groupby(by=['乳牛編號', '胎次']):
    dc[val[0]] = val[1]

In [14]:
for k, v in dc.items():
    print(v)

   ID  序列號  資料年度  資料月份 酪農場代號   乳牛編號    父親精液編號    母親乳牛編號            出生日期  胎次  \
0   1    1  2013     3     A  52612  507H7173  96040545  2011/1/1 00:00   1   
1   2    2  2013     4     A  52612  507H7173  96040545  2011/1/1 00:00   1   
2   3    3  2013     5     A  52612  507H7173  96040545  2011/1/1 00:00   1   
3   4    4  2013     6     A  52612  507H7173  96040545  2011/1/1 00:00   1   
4   5    5  2013     7     A  52612  507H7173  96040545  2011/1/1 00:00   1   
5   6    6  2013     8     A  52612  507H7173  96040545  2011/1/1 00:00   1   
6   7    7  2013    10     A  52612  507H7173  96040545  2011/1/1 00:00   1   
7   8    8  2013    11     A  52612  507H7173  96040545  2011/1/1 00:00   1   
8   9    9  2013    12     A  52612  507H7173  96040545  2011/1/1 00:00   1   
9  10   10  2014     1     A  52612  507H7173  96040545  2011/1/1 00:00   1   

   ...          最近分娩日期             採樣日期  月齡              檢測日期          最後配種日期  \
0  ...  2013/1/1 00:00  2013/3/20 00:00  26   201

[12 rows x 22 columns]
      ID  序列號  資料年度  資料月份 酪農場代號   乳牛編號    父親精液編號    母親乳牛編號             出生日期  \
176  177    1  2015     4     A  52621  507H8477  91040055  2011/2/18 00:00   
179  180    2  2015     5     A  52621  507H8477  91040055  2011/2/18 00:00   
178  179    3  2015     6     A  52621  507H8477  91040055  2011/2/18 00:00   
177  178    4  2015     7     A  52621  507H8477  91040055  2011/2/18 00:00   
181  182    5  2015     8     A  52621  507H8477  91040055  2011/2/18 00:00   
180  181    6  2015     9     A  52621  507H8477  91040055  2011/2/18 00:00   
182  183    7  2015    10     A  52621  507H8477  91040055  2011/2/18 00:00   
183  184    8  2015    11     A  52621  507H8477  91040055  2011/2/18 00:00   
184  185    9  2015    12     A  52621  507H8477  91040055  2011/2/18 00:00   
185  186   10  2016     1     A  52621  507H8477  91040055  2011/2/18 00:00   
189  190   11  2016     2     A  52621  507H8477  91040055  2011/2/18 00:00   
191  192   12  2016     3    

[24 rows x 22 columns]
      ID  序列號  資料年度  資料月份 酪農場代號   乳牛編號    父親精液編號    母親乳牛編號            出生日期  \
393  394    1  2013     7     A  52629  507H8747  96040505  2011/3/6 00:00   
394  395    2  2013     8     A  52629  507H8747  96040505  2011/3/6 00:00   
392  393    3  2013    10     A  52629  507H8747  96040505  2011/3/6 00:00   
391  392    4  2013    11     A  52629  507H8747  96040505  2011/3/6 00:00   
390  391    5  2013    12     A  52629  507H8747  96040505  2011/3/6 00:00   
386  387    6  2014     1     A  52629  507H8747  96040505  2011/3/6 00:00   
387  388    7  2014     2     A  52629  507H8747  96040505  2011/3/6 00:00   
389  390    8  2014     3     A  52629  507H8747  96040505  2011/3/6 00:00   
388  389    9  2014     4     A  52629  507H8747  96040505  2011/3/6 00:00   
383  384   10  2014     5     A  52629  507H8747  96040505  2011/3/6 00:00   
382  383   11  2014     6     A  52629  507H8747  96040505  2011/3/6 00:00   
384  385   12  2014     7     A  52629  5

[39 rows x 22 columns]
      ID  序列號  資料年度  資料月份 酪農場代號   乳牛編號    父親精液編號    母親乳牛編號             出生日期  \
620  621    1  2013     6     A  52636  501H9952  97040116  2011/3/27 00:00   
619  620    2  2013     7     A  52636  501H9952  97040116  2011/3/27 00:00   
618  619    3  2013     8     A  52636  501H9952  97040116  2011/3/27 00:00   
617  618    4  2013    10     A  52636  501H9952  97040116  2011/3/27 00:00   
616  617    5  2013    11     A  52636  501H9952  97040116  2011/3/27 00:00   
615  616    6  2013    12     A  52636  501H9952  97040116  2011/3/27 00:00   
613  614    7  2014     1     A  52636  501H9952  97040116  2011/3/27 00:00   
614  615    8  2014     2     A  52636  501H9952  97040116  2011/3/27 00:00   
611  612    9  2014     3     A  52636  501H9952  97040116  2011/3/27 00:00   
612  613   10  2014     4     A  52636  501H9952  97040116  2011/3/27 00:00   
623  624   11  2014     5     A  52636  501H9952  97040116  2011/3/27 00:00   
624  625   12  2014     6    

[7 rows x 22 columns]
      ID  序列號  資料年度  資料月份 酪農場代號   乳牛編號    父親精液編號    母親乳牛編號            出生日期  \
781  782    1  2017     7     A  52641  614H5010  97040119  2011/4/2 00:00   
780  781    2  2017     8     A  52641  614H5010  97040119  2011/4/2 00:00   
782  783    3  2017     9     A  52641  614H5010  97040119  2011/4/2 00:00   
789  790    4  2017    10     A  52641  614H5010  97040119  2011/4/2 00:00   
788  789    5  2017    11     A  52641  614H5010  97040119  2011/4/2 00:00   
786  787    6  2017    12     A  52641  614H5010  97040119  2011/4/2 00:00   
787  788    7  2018     1     A  52641  614H5010  97040119  2011/4/2 00:00   
784  785    8  2018     2     A  52641  614H5010  97040119  2011/4/2 00:00   
785  786    9  2018     3     A  52641  614H5010  97040119  2011/4/2 00:00   
783  784   10  2018     4     A  52641  614H5010  97040119  2011/4/2 00:00   

     胎次  ...          最近分娩日期              採樣日期  月齡              檢測日期  \
781   5  ...  2017/7/4 00:00    2017/7/5 00:00 

...

[23 rows x 22 columns]
        ID  序列號  資料年度  資料月份 酪農場代號     乳牛編號    父親精液編號    母親乳牛編號  \
1805  1806    1  2015     8     A  2051984  501H8784  99040442   
1806  1807    2  2015     9     A  2051984  501H8784  99040442   
1804  1805    3  2015    10     A  2051984  501H8784  99040442   
1808  1809    4  2015    11     A  2051984  501H8784  99040442   
1807  1808    5  2015    12     A  2051984  501H8784  99040442   
1809  1810    6  2016     1     A  2051984  501H8784  99040442   
1800  1801    7  2016     2     A  2051984  501H8784  99040442   
1802  1803    8  2016     3     A  2051984  501H8784  99040442   
1803  1804    9  2016     4     A  2051984  501H8784  99040442   
1801  1802   10  2016     5     A  2051984  501H8784  99040442   

                 出生日期  胎次  ...           最近分娩日期             採樣日期  月齡  \
1805  2013/1/30 00:00   1  ...  2015/7/13 00:00  2015/8/10 00:00  31   
1806  2013/1/30 00:00   1  ...  2015/7/13 00:00   2015/9/7 00:00  32   
1804  2013/1/30 00:00   1  ...  20

[10 rows x 22 columns]
        ID  序列號  資料年度  資料月份 酪農場代號     乳牛編號    父親精液編號    母親乳牛編號  \
1927  1928    1  2016     6     A  2051991  11H10079  95040486   
1925  1926    2  2016     7     A  2051991  11H10079  95040486   
1926  1927    3  2016     8     A  2051991  11H10079  95040486   
1928  1929    4  2016     9     A  2051991  11H10079  95040486   
1924  1925    5  2016    10     A  2051991  11H10079  95040486   
1914  1915    6  2016    11     A  2051991  11H10079  95040486   
1915  1916    7  2016    12     A  2051991  11H10079  95040486   
1913  1914    8  2017     1     A  2051991  11H10079  95040486   
1918  1919    9  2017     2     A  2051991  11H10079  95040486   
1917  1918   10  2017     3     A  2051991  11H10079  95040486   
1916  1917   11  2017     4     A  2051991  11H10079  95040486   
1923  1924   12  2017     5     A  2051991  11H10079  95040486   
1922  1923   13  2017     6     A  2051991  11H10079  95040486   
1920  1921   14  2017     7     A  2051991  11H10079 

[10 rows x 22 columns]
        ID  序列號  資料年度  資料月份 酪農場代號     乳牛編號  父親精液編號    母親乳牛編號            出生日期  \
2109  2110    1  2016    11     A  2052010  7H9222  98040660  2013/6/3 00:00   
2110  2111    2  2016    12     A  2052010  7H9222  98040660  2013/6/3 00:00   
2108  2109    3  2017     1     A  2052010  7H9222  98040660  2013/6/3 00:00   
2112  2113    4  2017     2     A  2052010  7H9222  98040660  2013/6/3 00:00   
2111  2112    5  2017     3     A  2052010  7H9222  98040660  2013/6/3 00:00   
2113  2114    6  2017     4     A  2052010  7H9222  98040660  2013/6/3 00:00   
2107  2108    7  2017     5     A  2052010  7H9222  98040660  2013/6/3 00:00   
2106  2107    8  2017     6     A  2052010  7H9222  98040660  2013/6/3 00:00   
2104  2105    9  2017     7     A  2052010  7H9222  98040660  2013/6/3 00:00   
2105  2106   10  2017     8     A  2052010  7H9222  98040660  2013/6/3 00:00   
2103  2104   11  2017     9     A  2052010  7H9222  98040660  2013/6/3 00:00   
2097  2098   12  

[21 rows x 22 columns]
        ID  序列號  資料年度  資料月份 酪農場代號     乳牛編號    父親精液編號 母親乳牛編號             出生日期  \
2274  2275    1  2019     7     A  2052017  501H9015  41337  2013/6/22 00:00   

      胎次  ...           最近分娩日期            採樣日期  月齡            檢測日期  最後配種日期  \
2274   3  ...  2019/6/25 00:00  2019/7/1 00:00  73  2019/7/2 00:00     NaN   

     最後配種精液 配種次數          前次分娩日期  第一次配種日期 第一次配種精液  
2274    NaN    0  2017/5/5 00:00      NaN     NaN  

[1 rows x 22 columns]

KeyboardInterrupt: 

In [ ]:
Train_ID = [rep_val for rep_val in report['ID'].values if rep_val not in submission['ID'].values]
Train_report = report.loc[report['ID'].isin(Train_ID)]
Train_Number = Train_report['乳牛編號'].values

In [ ]:
Test_ID = [rep_val for rep_val in report['ID'].values if rep_val in submission['ID'].values]
Test_report = report.loc[report['ID'].isin(Test_ID)]
Test_Number = Test_report['乳牛編號'].values

In [ ]:
report_drop_columns = ['資料年度', '資料月份', '父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Test_report_drop_columns = ['資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Train_report_x = Train_report.drop(columns=report_drop_columns, axis = 1)

Test_report_x = Test_report.drop(columns=Test_report_drop_columns, axis = 1)
Train_report_x

In [ ]:
# # birth feature
# birth_drop_columns = ['分娩日期', '乾乳日期', '犢牛編號1', '犢牛編號2', '登錄日期', '計算胎次', '犢牛體型', '犢牛性別', '酪農場代號']
# birth_x = birth.drop(columns=birth_drop_columns, axis = 1)

# Train_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Train_Number)]
# Test_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Test_Number)]
# Train_birth_x

# Merge Data

In [ ]:
x = lstm_x
# x = pd.merge(Train_report_x, Train_birth_x, on= ['乳牛編號'])

# x = pd.merge(x, Train_breed_x, on = ['乳牛編號'])
# x = pd.merge(x, Train_spec_x, on= ['乳牛編號'])
x = x.fillna(-1)
y = x['乳量'].values
x = x.drop(columns = ['ID', '乳牛編號','乳量'], axis = 1)

# x = normalize(x)
x

In [ ]:
test = pd.merge(Test_report_x, Test_birth_x, on =  ['乳牛編號'], how = 'left')

# test = pd.merge(test, Test_breed_x, on =  ['乳牛編號'], how = 'left')
# test = pd.merge(test, Test_spec_x, on =  ['乳牛編號'], how = 'left')
# test.fillna(0, inplace = True)

test_without_ID = test.drop(columns = ['乳牛編號','乳量','ID'], axis = 1)

# test_without_ID = normalize(test_without_ID)
# test_without_ID.fillna(0, inplace = True)
test_without_ID

In [ ]:
%run model.ipynb